# NLP with Julia for Non-English

This notebook is a tutorial on how to do NLP tasks in Julia
with a twist... We'll deal with non-english documents.
The main package for such task is `TextAnalysis.jl`. Yet, there are other functionalities that
are not present in Julia. So we use `PythonCall.jl` in order to sprinkle some python in our notebook.

In this tutorial, we are going to work with documents in Portuguese.

In [1]:
using Pkg
Pkg.activate(".")

using DataFrames
ENV["COLUMNS"]=2000
using VegaLite
using TextAnalysis
using TextModels

using CondaPkg
CondaPkg.add("nltk")
CondaPkg.add("spacy")
CondaPkg.add("ipython")
CondaPkg.add("cupy")
using PythonCall

  Activating project at `~/Main/EMAp/Julia_Tutorials/NLP`
    CondaPkg Found dependencies: /home/davibarreira/Main/EMAp/Julia_Tutorials/NLP/CondaPkg.toml
    CondaPkg Found dependencies: /home/davibarreira/.julia/packages/PythonCall/XUf6D/CondaPkg.toml
    CondaPkg Dependencies already up to date


In [2]:
using ProgressMeter

## 1. Importing a Corpus

To do some NLP analysis, we first need some texts. Hence, let's load a corpus in our notebook.

### 1.1 NLTK

We'll use python's nltk package, and the corpus of books from Machado de Assis, a brazilian writer.

In [3]:
nltk = pyimport("nltk")

Python module: <module 'nltk' from '/home/davibarreira/.local/lib/python3.10/site-packages/nltk/__init__.py'>

In [4]:
nltk_id = "machado"
nltk.download(nltk_id)

[nltk_data] Downloading package machado to
[nltk_data]     /home/davibarreira/nltk_data...
[nltk_data]   Package machado is already up-to-date!


Python bool: True

In [5]:
print(pyconvert(String, nltk.corpus.machado.readme()))
print(nltk.corpus.machado.fileids())

Machado de Assis -- Obra Completa

http://machado.mec.gov.br/

Public Domain

Contents:

Romance

romance/marm01.txt: Ressurreição (1872)
romance/marm02.txt: A Mão e a Luva (1874)
romance/marm03.txt: Helena (1876)
romance/marm04.txt: Iaiá Garcia (1878)
romance/marm05.txt: Memórias Póstumas de Brás Cubas (1881)
romance/marm06.txt: Casa Velha (1885)
romance/marm07.txt: Quincas Borba (1891)
romance/marm08.txt: Dom Casmurro (1899)
romance/marm09.txt: Esaú e Jacó (1904)
romance/marm10.txt: Memorial de Aires (1908)

Poesia

poesia/maps01.txt: Crisálidas (1864)
poesia/maps02.txt: Falenas (1870)
poesia/maps03.txt: Americanas (1875)
poesia/maps04.txt: Gazeta de Holanda (1886-88)
poesia/maps05.txt: Ocidentais (1901)
poesia/maps06.txt: O Almada (1908)
poesia/maps07.txt: Dispersas (1854-1939)

Contos

contos/macn001.txt: Contos Fluminenses (1870); Miss Dollar; Luís Soares; A mulher de preto; O segredo de Augusta; Confissões de uma viúva moça; Linha reta e linha curva; Frei Sim
contos/macn002.txt: 

In [6]:
# print(nltk.corpus.machado.raw("contos/macn076.txt"))

In [7]:
dom_casmurro = pyconvert(String,nltk.corpus.machado.raw("romance/marm08.txt"))[1:1000]

"Romance, Dom Casmurro, 1899\n\nDom Casmurro\n\nTexto de referência:\n\nObras Completas de Machado de Assis,\nvol. I,\n\nNova Aguilar, Rio de\nJaneiro, 1994.\n\n Publicado originalmente\npela Editora Garnier, Rio de Janeiro, 1899.\n\nCAPÍTULO PRIMEIRO\n\nDO TÍTULO\n\nUma noite destas, vindo da cidade\npara o Engenho Novo, encontrei no trem da Central um rapaz aqui do bairro, que\neu conheço de vista e de chapéu. Cumprimentou-me, sentou-se ao pé de mim, falou\nda Lua e dos ministros, e acabou recitando-me versos. A viagem era curta, e os\nversos pode ser que não fossem inteiramente maus. Sucedeu, porém, que, como eu\nestava cansado, fechei os olhos três ou quatro vezes; tanto bastou para que ele\ninterrompesse a leitura e metesse os versos no bolso.\n\n\u97 Continue, disse eu acordando.\n\n\u97 Já acabei, murmurou ele.\n\n\u97 São muito bonitos.\n\nVi-lhe fazer um gesto para\ntirá-los outra vez do bolso, mas não passou do gesto; estava amuado. No dia seguinte\nentrou a dizer de mim nome

## 1.2 Spacy

Next, let's use spacy, another python package.

In [8]:
spacy = pyimport("spacy")

Python module: <module 'spacy' from '/home/davibarreira/Main/EMAp/Julia_Tutorials/NLP/.CondaPkg/env/lib/python3.10/site-packages/spacy/__init__.py'>

Before going forward, we need to download the portuguese support in spacy.
The following lines of code runs the commnad `python -m ...` with the current Conda environment.

In [9]:
# CondaPkg.withenv() do
#   run(`python -m spacy download pt_core_news_lg`)
# end;

In [10]:
sentences = pyimport("spacy.lang.pt.examples" => "sentences")

sentences = pyconvert(Vector,sentences)

3-element Vector{String}:
 "Apple está querendo comprar uma startup do Reino Unido por 100 milhões de dólares"
 "Carros autônomos empurram a responsabilidade do seguro para os fabricantes.São Francisco considera banir os robôs de entrega que andam pelas calçadas"
 "Londres é a maior cidade do Reino Unido"

## 1.3 Creating our Corpus

We've showed how to get some samples using ntlk and spacy.
Let's now create our working corpus, which will consist of
the short-stories from Machado de Assis.

In [11]:
contos = [pyconvert(String,nltk.corpus.machado.raw(conto)) for conto in pyconvert(Vector,nltk.corpus.machado.fileids()) if startswith(conto,"contos/")];

These texts are too large. Let's truncate them.

In [12]:
contos = [first(conto,1000) for conto in contos]
print(contos[1])

Conto, Contos Fluminenses, 1870

Contos Fluminenses

Texto-fonte:

Obra Completa, Machado de Assis, vol. II,

Rio de Janeiro: Nova Aguilar, 1994.

Publicado originalmente pela
Editora Garnier, Rio de Janeiro, em 1870.

ÍNDICE

MISS DOLLAR

LUÍS
SOARES

A MULHER DE
PRETO

O
SEGREDO DE AUGUSTA

CONFISSÕES DE UMA VIÚVA MOÇA

LINHA
RETA E LINHA CURVA

FREI
SIMÃO

MISS
DOLLAR

ÍNDICE

Capítulo Primeiro

Capítulo II

Capítulo iii

Capítulo iv

Capítulo v

Capítulo vI

Capítulo vII

CAPÍTULO VIII

CAPÍTULO PRIMEIRO

Era conveniente ao romance que o leitor
ficasse muito tempo sem saber quem era Miss Dollar. Mas por outro lado,
sem a apresentação de Miss Dollar, seria o autor obrigado a longas
digressões, que encheriam o papel sem adiantar a ação. Não há hesitação
possível: vou apresentar-lhes Miss Dollar.

Se o leitor é rapaz e dado ao gênio
melancólico, imagina que Miss Dollar é uma inglesa pálida e delgada,
escassa de carnes e de sangue, abrindo à flor do rosto dois grandes olhos azuis
e sac

# 2. Tokens & Stopwords & Lemmatizaion/Stemmer

When working with texts in portuguese, we might want to go with spacy instead of nltk. The reason for this is that spacy already comes with support for portuguese, enabling us to easily do lemmatization. 

As of now (2022), Julia still does not have packages that implement such things for texts in portuguese. Again showing the necessity of integration python with ou Julia script.

**Now, let's tokenize our text**.

Tokenization can be thought of as splitting a text into words. The tricky thing is,
there are words such as "Palo Alto", which is actually a single name, hence,
we want to treat it as a single "token".

In [13]:
sent_tokenize, word_tokenize = pyimport("nltk.tokenize" =>
    ("sent_tokenize","word_tokenize"));

senttokenize(text) = pyconvert(Vector, sent_tokenize(text))
wordtokenize(text) = pyconvert(Vector, word_tokenize(text))

example = """
Uma noite destas, vindo da cidade para o Engenho Novo,
encontrei no trem da Central um rapaz aqui do bairro, que
eu conheço de vista e de chapéu. Cumprimentou-me, sentou-se ao pé de mim, falou
da Lua e dos ministros, e acabou recitando-me versos. A viagem era curta, e os
versos pode ser que não fossem inteiramente maus. Sucedeu, porém, que, como eu
estava cansado, fechei os olhos três ou quatro vezes; tanto bastou para que ele
interrompesse a leitura e metesse os versos no bolso.
"""

for (i,sentence) in enumerate(senttokenize(example))
    println(i,"--------------------------------------")
    println(sentence)
end

1--------------------------------------
Uma noite destas, vindo da cidade para o Engenho Novo,
encontrei no trem da Central um rapaz aqui do bairro, que
eu conheço de vista e de chapéu.
2--------------------------------------
Cumprimentou-me, sentou-se ao pé de mim, falou
da Lua e dos ministros, e acabou recitando-me versos.
3--------------------------------------
A viagem era curta, e os
versos pode ser que não fossem inteiramente maus.
4--------------------------------------
Sucedeu, porém, que, como eu
estava cansado, fechei os olhos três ou quatro vezes; tanto bastou para que ele
interrompesse a leitura e metesse os versos no bolso.


In [14]:
wordtokenize(example)

102-element Vector{String}:
 "Uma"
 "noite"
 "destas"
 ","
 "vindo"
 "da"
 "cidade"
 "para"
 "o"
 "Engenho"
 "Novo"
 ","
 "encontrei"
 ⋮
 "que"
 "ele"
 "interrompesse"
 "a"
 "leitura"
 "e"
 "metesse"
 "os"
 "versos"
 "no"
 "bolso"
 "."

It (almost) worked. Note that we got almost everything correct.
The only thing that seems wrong is the "Engenho Novo", which is a single
location, and thus, it should be a single token. Yet,
it's understandable that the code would not catch that.

Let's now try with spacy.

In [15]:
nlp = spacy.load("pt_core_news_lg");

In spacy, if we run `nlp(example)`, this will run a whole pipeline. If we just want the tokenizer, we can do
`nlt.tokenizer`.

In [16]:
doc = nlp.tokenizer(example)
tokens = [pyconvert(String,t.text) for t in pyconvert(Vector,collect(doc))]

109-element Vector{String}:
 "Uma"
 "noite"
 "destas"
 ","
 "vindo"
 "da"
 "cidade"
 "para"
 "o"
 "Engenho"
 "Novo"
 ","
 "\n"
 ⋮
 "\n"
 "interrompesse"
 "a"
 "leitura"
 "e"
 "metesse"
 "os"
 "versos"
 "no"
 "bolso"
 "."
 "\n"

The result from spacy is very similar to nltk. Yet, in this case,
we got additionally `\n`, the line break.

Now we can tokeninze our corpus. Let's tokenize the whole text.

In [17]:
files = [t for t in pyconvert(Vector,nltk.corpus.machado.fileids()) if startswith(t, "contos/")];

In [18]:
contos = [pyconvert(String,nltk.corpus.machado.raw(conto)) for conto in files]

function truncatetokens(text,maxtokens = 1000)
    doc = nlp.tokenizer(text)
    tokens = [pyconvert(String,t.text) for t in pyconvert(Vector,collect(doc))][begin:min(end,maxtokens)]
    # return join(tokens," ")
end

contos = @showprogress map(truncatetokens,contos);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:13


In [19]:
contos = join.(contos," ");

In [20]:
# nltk.download("stopwords")
stopwords = nltk.corpus.stopwords.words("portuguese");
stopwords = pyconvert(Vector,stopwords);

## 3. Part-of-speech Tagging and Dependency Parsing.

This task consists in tagging each word/token with it's grammatical morphology (e.g. subject, adverb, etc).

Let's start by running the spacy pipeline via the `nlp` function.

In [21]:
metacontos = @showprogress map(nlp,contos);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:13


Once we've done this, our original corpus has been enriched with metadata.
Let's put it in a DataFrame.

In [27]:
df = Dict(
    "file"=>[],
    "text"=>[],
    "pos"=>[],
    "pos_description"=>[],
    "dep"=>[],
    "dep_description"=>[],
    "lemma" =>[],
)
for (i,conto) in enumerate(metacontos)
    for token in conto
        push!(df["file"],files[i])
        push!(df["text"],pyconvert(String,token.text))
        push!(df["pos"],pyconvert(String,token.pos_))
        push!(df["pos_description"],pyconvert(String,spacy.explain(token.pos_)))
        push!(df["dep"],pyconvert(String,token.dep_))
        push!(df["lemma"],pyconvert(String,token.lemma_))
        try
            push!(df["dep_description"],pyconvert(String,spacy.explain(token.dep_)))
        catch
            push!(df["dep_description"],"")
        end
    end
end

df = DataFrame(df);

/home/davibarreira/Main/EMAp/Julia_Tutorials/NLP/.CondaPkg/env/lib/python3.10/site-packages/spacy/glossary.py:19: UserWarning: [W118] Term 'flat:name' not found in glossary. It may however be explained in documentation for the corpora used to train the language. Please check `nlp.meta["sources"]` for any relevant links.
  warnings.warn(Warnings.W118.format(term=term))
/home/davibarreira/Main/EMAp/Julia_Tutorials/NLP/.CondaPkg/env/lib/python3.10/site-packages/spacy/glossary.py:19: UserWarning: [W118] Term 'acl:relcl' not found in glossary. It may however be explained in documentation for the corpora used to train the language. Please check `nlp.meta["sources"]` for any relevant links.
  warnings.warn(Warnings.W118.format(term=term))
/home/davibarreira/Main/EMAp/Julia_Tutorials/NLP/.CondaPkg/env/lib/python3.10/site-packages/spacy/glossary.py:19: UserWarning: [W118] Term 'nsubj:pass' not found in glossary. It may however be explained in documentation for the corpora used to train the lang

In [28]:
filter(row->row[:pos_description] == "verb", df)

,dep,dep_description,file,lemma,pos,pos_description,text
,Any,Any,Any,Any,Any,Any,Any
1,ROOT,root,contos/macn001.txt,publicado,VERB,verb,Publicado
2,ROOT,root,contos/macn001.txt,vII,VERB,verb,vII
3,acl:relcl,,contos/macn001.txt,ficar,VERB,verb,ficasse
4,advcl,adverbial clause modifier,contos/macn001.txt,saber,VERB,verb,saber
5,acl,clausal modifier of noun (adjectival clause),contos/macn001.txt,obrigar,VERB,verb,obrigado
6,acl:relcl,,contos/macn001.txt,encher,VERB,verb,encheriam
7,advcl,adverbial clause modifier,contos/macn001.txt,adiantar,VERB,verb,adiantar
8,ROOT,root,contos/macn001.txt,haver,VERB,verb,há
9,parataxis,parataxis,contos/macn001.txt,apresentar-lhes,VERB,verb,apresentar-lhes


In [29]:
for chunk in nlp("São Paulo é uma bela cidade. Luís Soares é um morador de lá.").noun_chunks
    println(chunk)
end

São Paulo
Luís Soares


# 4.Named Entity Recognition

In [30]:
doc = nlp(sentences[1])

Python Doc: Apple está querendo comprar uma startup do Reino Unido por 100 milhões de dólares

In [31]:
for token in doc.ents
    println(token.text)
    println(token.label_)
    println(token.start_char,"-", token.end_char)
    println("-----------------------")
end

Apple
ORG
0-5
-----------------------
Reino Unido
LOC
43-54
-----------------------


# 5. Bag of Words